In [ ]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

# Converting a Non-Deterministic <span style="font-variant:small-caps;">Fsm</span> into a Deterministic <span style="font-variant:small-caps;">Fsm</span>

In this notebook we show how a non-deterministic <span style="font-variant:small-caps;">Fsm</span>
$$ F = \langle Q, \Sigma, \delta, q_0, A \rangle $$
can be transformed into a deterministic <span style="font-variant:small-caps;">Fsm</span> $\texttt{det}(F)$ such that both <span style="font-variant:small-caps;">Fsm</span>s accept the
same language, that is we have
$$ L(F) = L\bigl(\texttt{det}(F)\bigr). $$
The idea behind this transformation is that the <span style="font-variant:small-caps;">Fsm</span> $\texttt{det}(F)$ has to 
compute the set of all states that the <span style="font-variant:small-caps;">Fsm</span> $F$ could be in. 
Hence the states of the deterministic <span style="font-variant:small-caps;">Fsm</span> $\texttt{det}(F)$ are 
**sets** of states of the non-deterministic <span style="font-variant:small-caps;">Fsm</span> $F$.  A set of these states contains all those states that the non-deterministic <span style="font-variant:small-caps;">Fsm</span> 
$F$ could have reached.  Furthermore, a set $M$ of states of the <span style="font-variant:small-caps;">Fsm</span> $F$ is an accepting state of the <span style="font-variant:small-caps;">Fsm</span> $\texttt{det}(F)$ if the set $M$ contains an accepting state of the <span style="font-variant:small-caps;">Fsm</span> $F$.

## Declaring the Necessary Types

To implement the mathematical concepts of Finite Automata efficiently in TypeScript, we utilize the `RecursiveSet` library. This library allows us to treat sets as values: equality is determined by content, not reference. This is essential for the Subset Construction algorithm, where states of the DFA are sets of NFA states.

In [ ]:
import { RecursiveSet, Value } from "recursive-set";

### Basic Types

We define the fundamental building blocks of our automata:

1.  **Atomic States (`State`):**
    Identifiers for the states of the NFA (e.g., strings or numbers).

2.  **Input Symbols (`Char`):**
    Single characters from the alphabet $\Sigma$.

3.  **Composite States (`DFAState`):**
    For the **Subset Construction** algorithm, a state in the resulting DFA is defined as a **set** of NFA states. We define this explicit type alias to distinguish clearly between atomic states ($q \in Q_{NFA}$) and composite states ($S \subseteq Q_{NFA}$).

In [ ]:
type State = string | number;

type Char = string;

type DFAState = RecursiveSet<State>;

### Transition Map Keys

Mathematically, the transition function $\delta$ maps a pair $(q, c)$ to a target.
In JavaScript/TypeScript, `Map` keys are compared by reference. To allow efficient lookups for pairs, we use a **serialized key** representing the tuple $(q, c)$.

To ensure type safety and prevent the accidental usage of arbitrary strings as keys, we implement a **Branded Type**:
1.  We declare a `unique symbol` to create a "brand".
2.  We define `TransitionKey` as a string that carries this brand.
3.  We provide the universal helper function `key(q, c)`.

**The `key` Function:**
This function is the only legitimate way to create a `TransitionKey`. It is **polymorphic**, accepting both atomic states (for NFAs) and sets of states (for DFAs). It relies on the canonical string representation provided by `RecursiveSet`, ensuring that `{1, 2}` and `{2, 1}` result in the same key.

In [ ]:
declare const TransitionKeyBrand: unique symbol;

type TransitionKey = string & { [TransitionKeyBrand]: true };

function key(q: Value, c: Char): TransitionKey {
    return `${q.toString()},${c}` as TransitionKey;
}

### Transition Relations

We distinguish between the transition functions for NFAs and DFAs based on their codomain (return type).

**1. NFA Transition ($\delta_{NFA}$):**
The transition function maps a state and a character to a **set** of possible next states. This reflects the non-deterministic nature of the automaton.
$$\delta: Q \times \Sigma \to \mathcal{P}(Q)$$

In [ ]:
type TransRel = Map<TransitionKey, RecursiveSet<State>>;

**2. DFA Transition ($\delta_{DFA}$):**
The deterministic transition function maps a state and a character to exactly **one** next state.
$$\delta: Q_{DFA} \times \Sigma \to Q_{DFA}$$

In [ ]:
type TransRelDet = Map<TransitionKey, DFAState>;

### Automata Definitions

Finally, we define the object structures for the Non-Deterministic (NFA) and Deterministic (DFA) Finite Automata.

* **NFA:** A 5-tuple $A = (Q, \Sigma, \delta, q_0, F)$ where $\delta$ allows multiple targets.

In [ ]:
type NFA = {
    Q: RecursiveSet<State>;
    Sigma: RecursiveSet<Char>;
    delta: TransRel;
    q0: State;
    A: RecursiveSet<State>;
};

* **DFA:** A 5-tuple $A' = (Q', \Sigma, \delta', q'_0, F')$ where the states in $Q'$ are of type `DFAState` (sets).

In [ ]:
type DFA = {
    Q: RecursiveSet<DFAState>;
    Sigma: RecursiveSet<Char>;
    delta: TransRelDet;
    q0: DFAState;
    A: RecursiveSet<DFAState>;
};

<hr style="height:5px;background-color:blue">

## Subset Construction Algorithm

In order to present the construction of $\texttt{det}(F)$ we first have to define three auxiliary functions.

We start with the function `bigUnion`. Given a set `M` that contains sets of elements (i.e., a set of sets), the expression `bigUnion(M)` returns the union of all sets in `M`. Formally:
$$\texttt{bigUnion}(M) = \bigcup M = \bigl\{ x \bigm| \exists A \in M: x \in A \bigr\}.$$

In [ ]:
function bigUnion(sets: RecursiveSet<DFAState>): DFAState {
    const allElements: State[] = [];

    for (const subset of sets) {
        for (const elem of subset.raw) {
            allElements.push(elem);
        }
    }
    return RecursiveSet.fromArray(allElements);
}

We can test `bigUnion()` below:

In [ ]:
const input = new RecursiveSet<DFAState>(
    new RecursiveSet(1, 2, 3),
    new RecursiveSet(2, 3, 4),
    new RecursiveSet(3, 4, 5),
);

const result = bigUnion(input);

console.log("Result:", result.toString());
// Expected: {1, 2, 3, 4, 5}

The function `epsClosure` takes two arguments:
- `s` is a state, 
- `delta` is the transition function of the NFA $F$.

The function computes the set of all those states that can be reached from the state `s` via $\varepsilon$-transitions.
Formally, the set $\texttt{epsClosure}(s)$ is defined inductively:
- $s \in \texttt{epsClosure}(s)$.
- $p \in \texttt{epsClosure}(s) \wedge r \in \delta(p, \varepsilon) \;\rightarrow\; r \in \texttt{epsClosure}(s)$.
 
If the state $p$ is an element of the $\varepsilon$-closure of the state $s$ and there is an $\varepsilon$-transition from $p$ to some state $r$, then $r$ is also an element of the $\varepsilon$-closure of $s$.
 
The implementation of `epsClosure` uses a **fixed-point algorithm**. We start with $\{s\}$ and iteratively add reachable states until the set stabilizes.

In [ ]:
function epsClosure(s: State, delta: TransRel): RecursiveSet<State> {
    let result = new RecursiveSet<State>(s);

    while (true) {
        const newStatesParts = new RecursiveSet<DFAState>();

        for (const q of result) {
            const targets = delta.get(key(q, "ε"));
            if (targets) {
                newStatesParts.add(targets);
            }
        }
        const newStates = bigUnion(newStatesParts);

        if (newStates.isSubset(result)) {
            return result;
        }

        result = result.union(newStates);
    }
}

In order to transform a non-deterministic <span style="font-variant:small-caps;">Fsm</span> $F$ into a deterministic 
<span style="font-variant:small-caps;">Fsm</span>
$\texttt{det}(F)$ we have to extend the function $\delta:Q \times \Sigma \rightarrow 2^Q$ into the function
$$\widehat{\delta}: Q \times \Sigma \rightarrow 2^Q. $$
The idea is that given a state $q$ and a character $c$,  the value of $\widehat{\delta}(q,c)$ is the set of all states that the
<span style="font-variant:small-caps;">Fsm</span> $F$ could reach when it reads the character $c$ in state $q$ and then performs an arbitrary number of $\varepsilon$-transitions.  Formally, the definition of $\widehat{\delta}$ is as follows:
$$ \widehat{\delta}(q_1, c) := \bigcup \bigl\{ \texttt{epsClosure}(q_2) \bigm| q_2 \in \delta(q_1, c) \bigr \}. $$
This formula is to be read as follows:
- For every state $q_2 \in Q$ that can be reached from the state $q_1$ by reading the character $c$ we
  compute $\texttt{epsClosure}(q_2)$.
- Then we take the union of all these sets $\texttt{epsClosure}(q_2)$.

The function $\widehat{\delta}$ is implemented as the function `deltaHat`, which takes three arguments:
- `s` is a state,
- `c` is a character,
- `𝛿` is the transition function of a non-deterministic 
  <span style="font-variant:small-caps;">Fsm</span>.

This function computes the set of all those states that can be reached 
from `s` when we first have a transition from state `s` to some state `p` 
on reading the character `c` followed by any number of $\varepsilon$-transitions
starting in `p`.

In [ ]:
function deltaHat(s: State, c: Char, delta: TransRel): RecursiveSet<State> {
    const directTargets = delta.get(key(s, c));

    if (!directTargets || directTargets.isEmpty()) {
        return new RecursiveSet<State>();
    }

    const closures = new RecursiveSet<RecursiveSet<State>>();

    for (const q of directTargets) {
        closures.add(epsClosure(q, delta));
    }

    return bigUnion(closures);
}

The function $\widehat{\delta}$ maps a state into a set of states. Since the FSM $\texttt{det}(F)$ uses sets of states of the FSM $F$ as its states, we need a function that maps sets of states into sets of states. Hence we generalize the function $\widehat{\delta}$ to the function
$$\Delta: 2^Q \times \Sigma \rightarrow 2^Q$$
such that for a set $M$ of states and a character $c$ the expression $\Delta(M, c)$ computes the set of all those states that the FSM $F$ could be in if it is in a state from the set $M$, then reads the character $c$, and finally makes some $\varepsilon$-transitions.

The formal definition is as follows: 
$$\Delta(M,c) := \bigcup \bigl\{ \widehat{\delta}(q,c) \bigm| q \in M \bigr\}.$$

This formula is easy to understand: For every state $q \in M$ we compute the set of states that the FSM could be in after reading the character $c$ and doing some $\varepsilon$-transitions. Then we take the union of these sets.

In [ ]:
function capitalDelta(
    M: RecursiveSet<State>,
    c: Char,
    delta: TransRel,
): RecursiveSet<State> {
    const partials = new RecursiveSet<RecursiveSet<State>>();

    for (const q of M) {
        partials.add(deltaHat(q, c, delta));
    }

    return bigUnion(partials);
}

The function `allStates` takes three arguments:
- $Q$ is $\texttt{epsClosure}(q_0)$, where $q_0$ is the start state of the deterministic <span style="font-variant:small-caps;">Fsm</span> $\texttt{det}(F)$,
- $\delta$ is the transition function of the non-deterministic <span style="font-variant:small-caps;">Fsm</span> $F$, and
- $\Sigma$ is the alphabet of the non-deterministic <span style="font-variant:small-caps;">Fsm</span> $F$.

The function `allStates` computes the set of all states of the deterministic <span style="font-variant:small-caps;">Fsm</span> $\texttt{det}(F)$ that can be reached from the start state.

In [ ]:
function allStates(
    Q0: DFAState,
    delta: TransRel,
    Sigma: RecursiveSet<Char>,
): RecursiveSet<DFAState> {
    let result = new RecursiveSet<DFAState>(Q0);

    while (true) {
        const candidates: DFAState[] = [];

        for (const M of result) {
            for (const c of Sigma) {
                candidates.push(capitalDelta(M, c, delta));
            }
        }

        const newStates = RecursiveSet.fromArray(candidates);

        if (newStates.isSubset(result)) {
            return result;
        }

        result = result.union(newStates);
    }
}

Now we are ready to formally define how the deterministic <span style="font-variant:small-caps;">Fsm</span> $\texttt{det}(F)$
is computed from the non-deterministic <span style="font-variant:small-caps;">Fsm</span>
$F = \bigl\langle Q, \Sigma, \delta, q_0, A \bigr\rangle$.
We define: 
$$ \texttt{det}(F) := \bigl\langle \texttt{allStates}(\texttt{epsClosure}(q_0)), \Sigma, \Delta, \texttt{epsClosure}(q_0), \widehat{A} \bigr\rangle $$
where the components of this tuple are given as follows:
- The set of states of $\texttt{det}(F)$ is the set of all states that can be reached from the set $\texttt{epsClosure}(q_0)$.
- The input alphabet $\Sigma$ does not change when going from $F$ to $\texttt{det}(F)$.
  After all, the deterministic <span style="font-variant:small-caps;">Fsm</span> $\texttt{det}(F)$ has to recognize the same language as the non-deterministic
  <span style="font-variant:small-caps;">Fsm</span> $F$.
- The function $\Delta$, that has been defined previously, specified how the set of states change when a
  character is read.
- The start state $\texttt{epsClosure}(q_0)$ of the deterministic <span style="font-variant:small-caps;">Fsm</span> $\texttt{det}(F)$ is the set of all states
  that can be reached from the start state $q_0$ of the non-deterministic <span style="font-variant:small-caps;">Fsm</span> $F$
  via $\varepsilon$-transitions.
- The set of accepting states $\widehat{A}$ is the set of those subsets of $Q$ that contain an accepting
  state of the <span style="font-variant:small-caps;">Fsm</span> $F$:
  $$\widehat{A} := \bigl\{ M \in 2^Q \mid M \cap A \not= \{\} \bigl\}. $$
  
### Implementation Note: Lazy Construction
Although the powerset $2^Q$ can be exponentially large ($2^n$ states), we do not construct the entire powerset. Instead, our `allStates` function ensures we only construct the **reachable** part of the DFA. This makes the conversion efficient for many practical NFAs.

In [ ]:
function nfa2dfa(nfa: NFA): DFA {
    const { Sigma, delta, q0, A } = nfa;

    const newStart: DFAState = epsClosure(q0, delta);

    const newStates: RecursiveSet<DFAState> = allStates(newStart, delta, Sigma);

    const newDelta: TransRelDet = new Map();

    for (const M of newStates) {
        for (const c of Sigma) {
            const N = capitalDelta(M, c, delta);
            newDelta.set(key(M, c), N);
        }
    }

    const newFinalArr: DFAState[] = [];

    for (const M of newStates) {
        const intersection = M.intersection(A);

        if (!intersection.isEmpty()) {
            newFinalArr.push(M);
        }
    }

    const newFinal = RecursiveSet.fromArray(newFinalArr);

    return {
        Q: newStates,
        Sigma,
        delta: newDelta,
        q0: newStart,
        A: newFinal,
    };
}

To test this function, use the notebook `02-Test-NFA-2-DFA.ipynb`.